In [3]:
import pandas as pd
import openpyxl
import xlsxwriter
from datetime import date


In [36]:
input_df = pd.read_excel('../sample_files/Input_SD.xlsx')
input_df

,Targeting type,SKU,CODE,Market,PPC Type,Tactic,Match Type,BRAND,Date,PIC,STT,Budget,Targeting,Bid,Bid strategy
0,CONTEXTUAL VCPM,set 9 - 001,CC-0031-0002,US,SD,T00020,REACH,Chicnchill,20221212,HUY,1,15,"B09ZNJST22,B0BBF1MM2G",5.5,Dynamic bids - down only\n
1,CONTEXTUAL CPC,set 9 - 001,CC-0031-0002,US,SD,T00020,PAGE VISITS,Chicnchill,20221212,HUY,1,15,"B09ZNJST22,B0BBF1MM2G",7.6,Dynamic bids - down only\n
2,AUDIENCE CPC,set 9 - 001,CC-0031-0002,US,SD,T00030,CVR,Chicnchill,20221212,HUY,1,15,"category=""166164011""",20.0,Dynamic bids - down only\n
3,AUDIENCE VCPM,set 9 - 001,CC-0031-0002,US,SD,T00030,REACH,Chicnchill,20221212,HUY,1,15,"category=""166164011""",90.0,Dynamic bids - down only\n


<h2>Create Excel Form</h2>

In [5]:
def create_single_row(columns:list,cam_id):
    single_row_dictionary = {}
    for item in columns:
        if item == 'Product':
            single_row_dictionary[item] = 'Sponsored Products'
        elif item == 'Operation':
            single_row_dictionary[item] = 'Create'
        elif item == 'Campaign Id':
            single_row_dictionary[item] = cam_id
        elif item == 'State':
            single_row_dictionary[item] = 'enabled'
        else:
            single_row_dictionary[item] = None
    return single_row_dictionary

In [29]:
def createDataFrame(cam_id,budget,sku,bid_list:list,billing_strategy,date,tactic,targeting_list:list,match_type,targeting_type):
    columns = ['Product','Entity','Operation','Campaign Id','Ad Group Id','Campaign Name','Ad Group Name','Start Date','End Date','State','Tactic','Budget Type','Budget','Cost Type','Bid Optimization','SKU','ASIN','Ad Group Default Bid','Bid','Targeting Expression']
    df = pd.DataFrame(columns=columns)
    for i in range(0,3):
        df = df.append(create_single_row(columns=columns,cam_id=cam_id),ignore_index=True)
    
    df.iloc[0]['Entity'] = 'Campaign'
    df.iloc[0]['Campaign Name'] = cam_id
    df.iloc[0]['Start Date'] = date
    df.iloc[0]['Tactic'] = tactic
    df.iloc[0]['Budget Type'] = 'daily'
    df.iloc[0]['Budget'] = budget
    if targeting_type.lower()=='contextual vcpm' or targeting_type.lower()=='audience vcpm':
        df.iloc[0]['Cost Type'] = 'vCPM'
    elif targeting_type.lower()=='contextual cpc' or targeting_type.lower()=='audience cpc':
        df.iloc[0]['Cost Type'] = 'CPC'
    else:
        df.iloc[0]['Cost Type'] = ''

    df.iloc[1]['Entity'] = 'Ad group'
    df.iloc[1]['Ad Group Id'] = cam_id
    df.iloc[1]['Ad Group Name'] = cam_id
    df.iloc[1]['Ad Group Name'] = cam_id
    df.iloc[1]['Ad Group Default Bid'] = 1
    if match_type.lower() == 'reach':
        df.iloc[1]['Bid Optimization'] = 'reach'
    elif match_type.lower() =='page visits':
        df.iloc[1]['Bid Optimization'] = 'Optimize for page visits'
    elif match_type.lower() == 'cvr':
        df.iloc[1]['Bid Optimization'] = 'Optimize for conservations'
    else:
        df.iloc[1]['Bid Optimization'] = ''
    
    
    df.iloc[2]['Entity'] = 'Product ad'
    df.iloc[2]['Ad Group Id'] = cam_id
    df.iloc[2]['SKU'] = sku
    
    for i in range(len(targeting_list)):
        temp_row = create_single_row(columns=columns,cam_id=cam_id)
        temp_row['Entity'] = 'Product targeting'
        temp_row['Ad Group Id'] = cam_id
        if len(bid_list)>1:
            if i < len(bid_list):
                temp_row['Bid'] = bid_list[i]
            else:
                temp_row['Bid'] = bid_list[-1]
        else:
            temp_row['Bid'] = bid_list[0]
        if targeting_list[i].startswith("B0"):
            temp_row['Targeting Expression'] = '''asin="''' +str(targeting_list[i])+ '''"'''
        else:
            temp_row['Targeting Expression'] = str(targeting_list[i])
        df = df.append(temp_row,ignore_index=True)
    
    return df

In [30]:
createDataFrame_type1(cam_id='ABC123',budget=1000,sku='CC-100',bid_list=[5.5,6.5],billing_strategy='Bid downly',date='20221212',tactic='T00020',targeting_list=['BOProduct1','''category="BOproduct2"'''],match_type='page visits',targeting_type='contextual cpc')

,Product,Entity,Operation,Campaign Id,Ad Group Id,Campaign Name,Ad Group Name,Start Date,End Date,State,Tactic,Budget Type,Budget,Cost Type,Bid Optimization,SKU,ASIN,Ad Group Default Bid,Bid,Targeting Expression
0,Sponsored Products,Campaign,Create,ABC123,None,ABC123,None,20221212,None,enabled,T00020,daily,1000,CPC,None,None,None,None,NaN,None
1,Sponsored Products,Ad group,Create,ABC123,ABC123,None,ABC123,None,None,enabled,None,None,None,None,Optimize for page visits,None,None,1,NaN,None
2,Sponsored Products,Product ad,Create,ABC123,ABC123,None,None,None,None,enabled,None,None,None,None,None,CC-100,None,None,NaN,None
3,Sponsored Products,Product targeting,Create,ABC123,ABC123,None,None,None,None,enabled,None,None,None,None,None,None,None,None,5.5,BOProduct1
4,Sponsored Products,Product targeting,Create,ABC123,ABC123,None,None,None,None,enabled,None,None,None,None,None,None,None,None,6.5,"category=""BOproduct2"""


In [37]:
input_df

,Targeting type,SKU,CODE,Market,PPC Type,Tactic,Match Type,BRAND,Date,PIC,STT,Budget,Targeting,Bid,Bid strategy
0,CONTEXTUAL VCPM,set 9 - 001,CC-0031-0002,US,SD,T00020,REACH,Chicnchill,20221212,HUY,1,15,"B09ZNJST22,B0BBF1MM2G",5.5,Dynamic bids - down only\n
1,CONTEXTUAL CPC,set 9 - 001,CC-0031-0002,US,SD,T00020,PAGE VISITS,Chicnchill,20221212,HUY,1,15,"B09ZNJST22,B0BBF1MM2G",7.6,Dynamic bids - down only\n
2,AUDIENCE CPC,set 9 - 001,CC-0031-0002,US,SD,T00030,CVR,Chicnchill,20221212,HUY,1,15,"category=""166164011""",20.0,Dynamic bids - down only\n
3,AUDIENCE VCPM,set 9 - 001,CC-0031-0002,US,SD,T00030,REACH,Chicnchill,20221212,HUY,1,15,"category=""166164011""",90.0,Dynamic bids - down only\n


In [40]:
frameslist=[]
for i in range(len(input_df)):
    cam_id = str(input_df.iloc[i]['SKU']) +' ' + str(input_df.iloc[i]['CODE']) + ' ' + str(input_df.iloc[i]['Market']) +' ' +  str(input_df.iloc[i]['PPC Type']) + ' ' + str(input_df.iloc[i]['Tactic']) + ' ' + str(input_df.iloc[i]['Match Type']) +' ' +  str(input_df.iloc[i]['BRAND']) +' ' +  str(input_df.iloc[i]['Date'])+' ' + str(input_df.iloc[i]['PIC'])+' ' + str(input_df.iloc[i]['STT'])
    bid_list = str(input_df.iloc[i]['Bid']).split(',')
    targeting_list = str(input_df.iloc[i]['Targeting']).split(',')
    print(bid_list)
    print(targeting_list)
    temp_df = createDataFrame(cam_id = cam_id,budget=str(input_df.iloc[i]['Budget']),sku=str(input_df.iloc[i]['SKU']),bid_list=bid_list,
                                    billing_strategy=str(input_df.iloc[i]['Bid strategy']),date=str(input_df.iloc[i]['Date']),tactic=str(input_df.iloc[i]['Tactic']),
                                    targeting_list=targeting_list,match_type=str(input_df.iloc[i]['Match Type']),targeting_type=str(input_df.iloc[i]['Targeting type']))
    frameslist.append(temp_df)

output_df = pd.concat(frameslist)

['5.5']
['B09ZNJST22', 'B0BBF1MM2G']
['7.6']
['B09ZNJST22', 'B0BBF1MM2G']
['20.0']
['category="166164011"']
['90.0']
['category="166164011"']


In [41]:
output_df

,Product,Entity,Operation,Campaign Id,Ad Group Id,Campaign Name,Ad Group Name,Start Date,End Date,State,Tactic,Budget Type,Budget,Cost Type,Bid Optimization,SKU,ASIN,Ad Group Default Bid,Bid,Targeting Expression
0,Sponsored Products,Campaign,Create,set 9 - 001 CC-0031-0002 US SD T00020 REACH Ch...,None,set 9 - 001 CC-0031-0002 US SD T00020 REACH Ch...,None,20221212,None,enabled,T00020,daily,15,vCPM,None,None,None,None,None,None
1,Sponsored Products,Ad group,Create,set 9 - 001 CC-0031-0002 US SD T00020 REACH Ch...,set 9 - 001 CC-0031-0002 US SD T00020 REACH Ch...,None,set 9 - 001 CC-0031-0002 US SD T00020 REACH Ch...,None,None,enabled,None,None,None,None,reach,None,None,1,None,None
2,Sponsored Products,Product ad,Create,set 9 - 001 CC-0031-0002 US SD T00020 REACH Ch...,set 9 - 001 CC-0031-0002 US SD T00020 REACH Ch...,None,None,None,None,enabled,None,None,None,None,None,set 9 - 001,None,None,None,None
3,Sponsored Products,Product targeting,Create,set 9 - 001 CC-0031-0002 US SD T00020 REACH Ch...,set 9 - 001 CC-0031-0002 US SD T00020 REACH Ch...,None,None,None,None,enabled,None,None,None,None,None,None,None,None,5.5,"asin=""B09ZNJST22"""
4,Sponsored Products,Product targeting,Create,set 9 - 001 CC-0031-0002 US SD T00020 REACH Ch...,set 9 - 001 CC-0031-0002 US SD T00020 REACH Ch...,None,None,None,None,enabled,None,None,None,None,None,None,None,None,5.5,"asin=""B0BBF1MM2G"""
0,Sponsored Products,Campaign,Create,set 9 - 001 CC-0031-0002 US SD T00020 PAGE VIS...,None,set 9 - 001 CC-0031-0002 US SD T00020 PAGE VIS...,None,20221212,None,enabled,T00020,daily,15,CPC,None,None,None,None,None,None
1,Sponsored Products,Ad group,Create,set 9 - 001 CC-0031-0002 US SD T00020 PAGE VIS...,set 9 - 001 CC-0031-0002 US SD T00020 PAGE VIS...,None,set 9 - 001 CC-0031-0002 US SD T00020 PAGE VIS...,None,None,enabled,None,None,None,None,Optimize for page visits,None,None,1,None,None
2,Sponsored Products,Product ad,Create,set 9 - 001 CC-0031-0002 US SD T00020 PAGE VIS...,set 9 - 001 CC-0031-0002 US SD T00020 PAGE VIS...,None,None,None,None,enabled,None,None,None,None,None,set 9 - 001,None,None,None,None
3,Sponsored Products,Product targeting,Create,set 9 - 001 CC-0031-0002 US SD T00020 PAGE VIS...,set 9 - 001 CC-0031-0002 US SD T00020 PAGE VIS...,None,None,None,None,enabled,None,None,None,None,None,None,None,None,7.6,"asin=""B09ZNJST22"""
4,Sponsored Products,Product targeting,Create,set 9 - 001 CC-0031-0002 US SD T00020 PAGE VIS...,set 9 - 001 CC-0031-0002 US SD T00020 PAGE VIS...,None,None,None,None,enabled,None,None,None,None,None,None,None,None,7.6,"asin=""B0BBF1MM2G"""


In [42]:
def export_excel(output_df:pd.DataFrame,file_export_path='',sheet_name=''):
    writer = pd.ExcelWriter(file_export_path, engine='xlsxwriter')
    output_df.to_excel(writer,sheet_name = sheet_name, index=False)
    # output_df.to_excel(writer,sheet_name = 'sheet_keyword', index=False)
    writer.save() 
export_excel(output_df=output_df,file_export_path='text.xlsx',sheet_name='Sheet1')